In [ ]:
using Transformation, Plots, Logging
disable_logging(LogLevel(1000))
include("graphics.jl");

# Нотация Денавита и Хартенберга

Нотация Денавита и Хартенберга позволяет описать любую последовательную кинематическую схему.
В основе этого подхода лежит соображение о том, что оси двух соседних кинематических пар всегда зафиксированны друг относительно друга.
Тогда мы можем использовать подход Плюккера для описания взаимного расположения прямых, совпадающих с этими осями.

Пускай даны две прямые $A$ и $B$, точка $p$, лежащая на $A$ и единичный вектор $n$, перпендикулярный $A$.
Договоримся что у прямой есть направление (относительно $p$ можно смещаться как в положительную, так и в отрицательную сторону вдоль $A$).
Тогда чтобы описать положение прямой $B$ относительно $A$ и $p$ нужно задать:
- смещение вдоль $A$ на величину $t_z$
- поворот вектора $n$ вокруг $A$ на угол $r_z$
- смещение вдоль повернутого вектора $n$ на $t_x$
- поворот вокруг повернутого вектора $n$ на угол $r_z$

Этот набор операций соответсвует
- смещению вдоль $A$ от точки $p$ до общего перпендикуляра (или точки пересечения) $A$ и $B$
- повороту в сторону этого общего перпендикуляра
- смещению вдоль этого общего перпендикуляра
- повороту вокруг этого общего перпендикуляра

In [ ]:
function dh(tz::Real, rz::Real, tx::Real, rx::Real)::Transf
    Transf(Vec(0, 0, tz), Quat(rz, Vec(0, 0, 1))) + Transf(Vec(tx, 0, 0), Quat(rx, Vec(1, 0, 0)))
end;

Зададим протой трехзвенный манипулятор в нотации Денавита-Хартенберга

In [ ]:
function three(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    q = map(deg2rad, q)
    t0 = Transf()
    t1 = t0 + dh(l[1], q[1] + pi/2, 0, pi/2)
    t2 = t1 + dh(0, q[2] + pi/2, l[2], 0)
    t3 = t2 + dh(0, q[3], l[3], 0)
    [t0, t1, t2, t3]
end;

И рассмотрим решение прямой задачи кинематики.
Особое внимание стоит уделить тому, что вращение всегда происходит вокруг локальных осей $Z$

In [ ]:
l = [3, 3, 2];

@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12)
    )
    
    q = [
        90sin(2pi * t),
        45 + 45cos(2pi * t),
        60 + 30sin(4pi * t + pi/3)
    ]
    chain = three(l, q)
    for segment in chain
        plotAxis!(p, segment, 0.5)
    end
    plotChain!(p, chain)
end

Идентично описывается манипулятор кинематики PUMA

In [ ]:
function puma(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    q = map(deg2rad, q)
    t0 = Transf()
    t1 = t0 + dh(l[1], q[1] + pi/2, 0, pi/2)
    t2 = t1 + dh(0, q[2] + pi/2, l[2], 0)
    t3 = t2 + dh(0, q[3] + pi/2, 0, pi/2)
    t4 = t3 + dh(l[3], q[4], 0, -pi/2)
    t5 = t4 + dh(0, q[5], 0, pi/2)
    t6 = t5 + dh(l[4], q[6], 0, 0)
    
    [t0, t1, t2, t3, t4, t5, t6]
end;

In [ ]:
l = [4, 5, 4, 1];

qs = [0, 45, 90, 90, 90, 0]
qe = [120, -15, 120, 60, -90, 360]

pathX = []
pathY = []
pathZ = []
@gif for t = 0:.01:1
    p = plot(xlims = (-6, 6), ylims = (-6, 6), zlims = (0, 12))
    
    q = qs + t * (qe - qs)
    chain = puma(l, q)
    lst = last(chain)
    push!(pathX, lst.v.x); push!(pathY, lst.v.y); push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    for joint in chain
        plotAxis!(p, joint, 0.5)
    end
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end